# Module 12: Maps

Let's draw some maps. 🗺🧐

Due to some issues with Bokeh vs. Jupyter Lab, please use Jupyter notebook for the later part. Make sure you have the most recent version of everything: https://altair-viz.github.io/getting_started/installation.html#quick-start-altair-notebook and have the vega nbextension: https://github.com/vega/ipyvega#install-and-run

Notes:

1. If Altair gives errors such as `Javascript error: unable to ...`, it is possibly due to the browser blocking certain Javascript features. Changing to another browser may solve this problem. Example: if you use Chrome, you may want to use Safari instead.

2. If you use Jupyter notebook, you need to add this line:

`alt.renderers.enable('notebook')`

However, if you use Jupyter lab, do **not** run this line.

## A dotmap with Altair

Let's start with altair. 

In [ ]:
import altair as alt

# saving data into a file rather than embedding into the chart
alt.data_transformers.enable('json') 

# jupyter notebook needs this option.
alt.renderers.enable('notebook')

### A data cleaning lesson

Maybe we need a dataset with geographical coordinates. This `zipcodes` dataset contains the location and zipcode of each zip code area. 

In [ ]:
from vega_datasets import data

zipcodes_url = data.zipcodes.url
zipcodes = data.zipcodes()
zipcodes.head()

There is a critical issue with this data. Can you spot it? ZIP codes are supposed to be 5 digits right? But why do they have only three digits? Let's sample more rows. 

In [ ]:
zipcodes.sample(5)

Ok, some have five digits but some have fewer. Let's check the dtype. 

In [ ]:
zipcodes.zip_code.dtype

Ah, they are stored as integer! I know that there are zipcodes starting with '0', so they must have been chopped off. Let's load the data again, this time by specifying the correct dtype. 

In [ ]:
zipcodes = data.zipcodes(dtype={'zip_code': 'category'})
zipcodes.head()

Indeed! 

In [ ]:
zipcodes.zip_code.dtype

In [ ]:
zipcodes.zip_code.apply(len).value_counts()

Yes, they all have five digits now. I think this is a useful reminder about the importance of being aware of data types, and being critical about your data. (btw, I've [reported this](https://github.com/altair-viz/vega_datasets/issues/16)). 

Btw, you'll have fewer issues if you pass URL instead of a dataframe to `alt.Chart`. 

### Let's draw it

Now we have the dataset loaded and start drawing some plots. Let's say you don't know anything about map projections. What would you try with geographical data? The simplest thing is probably considering (longitude, latitude) as a Cartesian coordinate and directly plot them. 

In [ ]:
alt.Chart(zipcodes_url).mark_circle().encode(
    x='longitude:Q',
    y='latitude:Q',
)

Actually this itself is a map projection called [Equirectangular projection](https://en.wikipedia.org/wiki/Equirectangular_projection). This projection (or almost a *non-projection*) is super straight-forward and doesn't require any processing of the data. So, often it is used to just quickly explore geographical data. As you dig deeper, you still want to think about which map projection fits your need best. Don't just use equirectangular projection without any thoughts! 

Anyway, let's make it look slighly better by reducing the size of the circles and adjusting the aspect ratio. 

In [ ]:
alt.Chart(zipcodes_url).mark_circle(size=2).encode(
    x='longitude:Q',
    y='latitude:Q',
).properties(
    width=700,
    height=200,
)

But, a much better way to do this is explicitly specifying that they are lat, lng coordinates by using `longitude=` and `latitude=`, rather than `x=` and `y=`. If you do that, altair automatically adjust the aspect ratio. Try to change the width and height. 

In [ ]:
alt.Chart(zipcodes_url).mark_circle(size=2).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
).properties(
    width=700,
    height=300,
)

Because the [American empire is far-reaching and complicated](https://www.youtube.com/watch?v=ASSOQDQvVLU), the information density of this map is very low (although interesting). A common projection for visualizing US data is [AlbersUSA](https://bl.ocks.org/mbostock/5545680), which uses [Albers (equal-area) projection](https://en.wikipedia.org/wiki/Albers_projection). This is a standard projection used in United States Geological Survey and the United States Census Bureau. Albers USA is simply a composition of US main land, Alaska, and Hawaii. 

To use it, we simply call `project` method and specify which variables are `longitude` and `latitude`. 

In [ ]:
alt.Chart(zipcodes_url).mark_circle(size=2).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
).project(
    type='albersUsa'
).properties(
    width=700,
    height=400,
)

Can we see the large-scale division of the zipcodes? We can use the fact that the zipcodes are hierarchically organized. That is, the first digit captures the largest area divisions. 

Altair provides some data transformation functionalities. One of them is extracting a substring from a variable. 

In [ ]:
from altair.expr import datum, substring

alt.Chart(zipcodes_url).mark_circle(size=2).transform_calculate(
    'first_digit', substring(datum.zip_code, 0, 1)
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='first_digit:N',
).project(
    type='albersUsa'
).properties(
    width=700,
    height=400,
)

For each row (`datum`), you obtain the `zip_code` variable and get the substring (imagine Python list slicing), and then you call the result `first_digit`. Now, you can use this `first_digit` variable to color the circles. Also note that we specify `first_digit` as a *nominal* variable, not quantitative, to obtain a categorical colormap. But we can also play with it too. 

**Q: Why don't you extract the first two digits, name it as `two_digits`, and declare that as a quantitative variable? Any interesting patterns?** 

In [ ]:
# Implement


Btw, we can also put a tooltip. Also, you can always click "view source" or "open in Vega Editor" to look at the json object that **defines** this visualization. You can embed this json object on your webpage and easily put up an interactive visualization. 

In [ ]:
alt.Chart(zipcodes_url).mark_circle(size=2).transform_calculate(
    'first_digit', substring(datum.zip_code, 0, 1)
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='first_digit:N',
    tooltip='zip_code:N'
).project(
    type='albersUsa'
).properties(
    width=700,
    height=400,
)

## Choropleth 

Let's try some choropleth now. Vega datasets have US county / state boundary data (`us_10m`) and world country boundary data (`world-110m`). You can take a look at the boundaries on GitHub (they renders topoJSON files):

- https://github.com/vega/vega-datasets/blob/gh-pages/data/us-10m.json
- https://github.com/vega/vega-datasets/blob/gh-pages/data/world-110m.json

If you click "Raw" then you can take a look at the actual file, which is hard to read. 

Essentially, each file is a large dictionary with the following keys. 

In [ ]:
usmap = data.us_10m()
usmap.keys()

In [ ]:
usmap['type']

In [ ]:
usmap['transform']

This `transformation` is used to *quantize* the data and store the coordinates in integer (easier to store than float type numbers). 

https://github.com/topojson/topojson-specification#212-transforms

In [ ]:
usmap['objects'].keys()

This data contains not only county-level boundaries (objects) but also states and land boundaries. 

In [ ]:
usmap['objects']['land']['type'], usmap['objects']['states']['type'], usmap['objects']['counties']['type']

`land` is a multipolygon (one object) and `states` and `counties` contains many geometrics (multipolygons) because there are many states (counties). We can look at a state as a set of arcs that define it. It's `id` captures the identity of the state and is the key to link to other datasets. 

In [ ]:
state1 = usmap['objects']['states']['geometries'][1]
state1

The `arcs` referred here is defined in `usmap['arcs']`. 

In [ ]:
usmap['arcs'][:10]

It seems pretty daunting to work with this dataset, right? But fortunately people have already built tools to handle such data. 

In [ ]:
states = alt.topo_feature(data.us_10m.url, 'states')

In [ ]:
states

In [ ]:
alt.Chart(states).mark_geoshape().properties(
    width=500,
    height=300
)

We have a map of US states. :) Can you use AlbersUSA projection on this? 

In [ ]:
# Implement


Can you do the same thing with counties and draw county boundaries?

In [ ]:
# Implement


Let's load some county-level unemployment data. 

In [ ]:
unemp_data = data.unemployment(sep='\t')
unemp_data.head()

This dataset has unemployment rate. When? I don't know. We don't care about data provenance here because the goal is quickly trying out choropleth. But if you're working with a real dataset, you should be very sensitive about the provenance of your dataset. Make sure you understand where the data came from and how it was processed. 

Anyway, for each county specified with `id`. To combine two datasets, we use "Lookup transform" - https://vega.github.io/vega/docs/transforms/lookup/. Essentially, we use the `id` in the map data to look up (again) `id` field in the `unemp_data` and then bring in the `rate` variable. Then, we can use that `rate` variable to encode the color of the `geoshape` mark. 

In [ ]:
alt.Chart(us_counties).mark_geoshape().project(
    type='albersUsa'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(unemp_data, 'id', ['rate'])
).encode(
    color='rate:Q'
).properties(
    width=700,
    height=400
)

There you have it, a basic choropleth map. 😎 


## Raster visualization with datashader

Although many geovisualizations use vector graphics, raster visualization is still useful especially when you deal with images and lots of datapoints. Datashader is a package that aggregates and visualizes a large amount of data very quickly. Given a *scene* (visualization boundary, resolution, etc.), it quickly aggregate the data and produce **pixels** and send them to you. 

To appreciate its power, we need a fairly large dataset. Let's use NYC taxi trip dataset on Kaggle: https://www.kaggle.com/kentonnlp/2014-new-york-city-taxi-trips You can download even bigger trip data from NYC open data website: https://opendata.cityofnewyork.us/data/

Ah, and you want to install the datashader, bokeh, and holoviews first if you don't have them yet.  

    pip install datashader bokeh holoviews

or 

    conda install datashader bokeh holoviews
    

In [1]:
import pandas as pd
import datashader as ds
from datashader import transfer_functions as tf
from colorcet import fire

ModuleNotFoundError: No module named 'datashader'

Because the dataset is pretty big, let's use a small sample first (`nrows=10000`). For this visualization, we only keep the dropoff location. 

In [ ]:
nyctaxi = pd.read_csv('~/Downloads/nyc_taxi_data_2014.csv', 
                      nrows=10000, 
                      usecols=['dropoff_longitude', 'dropoff_latitude'])
nyctaxi.head()

Although the dataset is different, we can still follow the example here: http://datashader.org/getting_started/1_Introduction.html

In [ ]:
agg = ds.Canvas().points(nyctaxi, 'dropoff_longitude', 'dropoff_latitude')
tf.set_background(tf.shade(agg, cmap=fire),"black")

Why can't we see anything? Wait, do you see the small dots on the left top? Can that be New York City? Maybe we don't see anything because some people travel very far? or because the dataset has some missing data?

In [ ]:
nyctaxi.isna().sum()

In [ ]:
nyctaxi.dropna(inplace=True)

Now there's no NaN in the data. let's try histogram. 

In [ ]:
%matplotlib inline
nyctaxi.hist()

Ah, there seems to be some outliers, which look like zero. Because lat lng should vary within a very small range, let's filter the data.  

In [ ]:
df = nyctaxi[nyctaxi['dropoff_latitude'].between(40.5, 41) & nyctaxi['dropoff_longitude'].between(-74.1, -73.7)]

In [ ]:
df.hist()

Now the histograms look pretty reasonable. 

In [ ]:
agg = ds.Canvas().points(df, 'dropoff_longitude', 'dropoff_latitude')
tf.set_background(tf.shade(agg, cmap=fire), "black")

Do you see the black empty space at the center? That looks like the Central Park. This is cool, but it'll be awesome if we can explore the data interactively. 

Datashader can work with `holoviews` and `bokeh` libraries to enable some interactivity. See http://holoviews.org/gallery/apps/bokeh/nytaxi_hover.html#bokeh-gallery-nytaxi-hover But, we'll not try to reproduce this because it's a bit too much work, and because, at this moment, it's not easy to install `geoviews` and `cartopy`. Let's do a simpler version. First import the libraries and set it up to use bokeh, which is like a backend for drawing objects with javascript. 

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import datashade
hv.extension('bokeh')

In [ ]:
%%output size=200

points = hv.Points(df, ['dropoff_longitude', 'dropoff_latitude'])
taxi_trips = datashade(points, cmap=fire)
taxi_trips

Enable dragging (four-directional arrows) and wheel-zoom (a scroll wheel and a magnifying glass) on the right side of the figure and explore the data by dragging and zooming in and out. 

Pretty neat, right? 

Ok, now let's get serious by loading the whole dataset (it may take some time btw). Drop NaN rows, and filter using the latitude and longitude range as before.

In [ ]:
# Implement 


Can you feed the data directly to datashader to reproduce the static plot, this time with the full data?

In [ ]:
# Implement


Wow, that's fast. Also it looks cool! 

Now with the holoviews. 

In [ ]:
# Implement


**Q: how many rows (data points) are we visualizing right now?**

In [ ]:
# figure it out

We're working with ?? rows (points) right now. Yet, datashader + holoviews + bokeh renders everything almost in real time! 

## map overlays

Another useful types of visualization is overlaying data on an explorable map (Google maps, Open streetmap, ...). [Leaflet.js](https://leafletjs.com) is one of the easiest options to do that on the web, and there is a Python bridge of it: https://github.com/jupyter-widgets/ipyleaflet  Although we will not go into details, it's certainly something that's worth checking out if you're using geographical data. 